# Assistants API - 파일 검색

파일 검색은 독점 제품 정보나 사용자가 제공한 문서 등 모델 외부의 지식으로 도우미를 강화합니다. OpenAI는 자동으로 문서를 구문 분석 및 청크하고, 임베딩을 생성 및 저장하며, 벡터 및 키워드 검색을 모두 사용하여 관련 콘텐츠를 검색하여 사용자 쿼리에 응답합니다.  

이 실습에서는 회사의 재무제표에 대한 질문에 답변하는 데 도움이 되는  assistant를 만듭니다.

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # read local .env file

True

In [2]:
from openai import OpenAI
client = OpenAI()

Model = "gpt-4o-mini"

### 1단계: 파일 검색이 활성화된 새 assistant 만들기
Assistants tools parameter수에서 file_search가 활성화된 새 assistant를 만듭니다  
file_search 도구가 활성화되면 모델은 사용자 메시지를 기반으로 콘텐츠를 검색할 시기를 결정합니다.

In [3]:
# assistant 생성
assistant = client.beta.assistants.create(
  name="Financial Analyst Assistant",
  instructions="당신은 전문 재무 분석가입니다. 지식 기반을 사용하여 감사된 재무제표에 대한 질문에 답하십시오.",
  model=Model,
  tools=[{"type": "file_search"}],
)

assistant

Assistant(id='asst_XedEg8jmLxrtK3z8fvf241HD', created_at=1738906598, description=None, instructions='당신은 전문 재무 분석가입니다. 지식 기반을 사용하여 감사된 재무제표에 대한 질문에 답하십시오.', metadata={}, model='gpt-4o-mini', name='Financial Analyst Assistant', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=[])), top_p=1.0, reasoning_effort=None)

### 2단계: Vector Store 생성

In [4]:
# "Financial Statements"라는 벡터 스토어 생성
vector_store = client.beta.vector_stores.create(name="Financial Statements")
 
# OpenAI에 업로드할 파일 준비
file_paths = ["edgar/goog-10k.pdf", "edgar/brka-10k.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
 
# 업로드 및 폴링 SDK 도우미를 사용하여 파일을 업로드하고 벡터 스토어에 추가,
# 파일 배치의 완료 상태를 폴링합니다.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# 이 작업의 결과를 보기 위해 상태 및 파일 개수를 출력할 수 있습니다.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


### 3단계: 새로운 Vector Store를 사용하도록 어시스턴트 업데이트
어시스턴트가 파일에 액세스할 수 있도록 하려면 어시스턴트의 tool_resources를 새 vector_store ID로 업데이트합니다.

In [5]:
#기존 Assistant를 업데이트하여 파일 검색 기능을 사용할 수 있도록 설정
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,    # 업데이트할 Assistant의 ID
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},  # 파일 검색을 위한 벡터 저장소 추가
)
assistant

Assistant(id='asst_XedEg8jmLxrtK3z8fvf241HD', created_at=1738906598, description=None, instructions='당신은 전문 재무 분석가입니다. 지식 기반을 사용하여 감사된 재무제표에 대한 질문에 답하십시오.', metadata={}, model='gpt-4o-mini', name='Financial Analyst Assistant', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_67a59becb1d88191a62b5b60faaac064'])), top_p=1.0, reasoning_effort=None)

### 4단계: 스레드 만들기
스레드에 메시지 첨부 파일로 파일을 첨부할 수도 있습니다. 이렇게 하면 스레드와 연결된 또 다른 벡터 저장소가 생성됩니다. 또는 이 스레드에 이미 연결된 벡터 저장소가 있는 경우 새 파일을 기존 스레드 벡터 저장소에 연결합니다. 이 스레드에서 실행을 생성하면 파일 검색 도구는 어시스턴트의 vector_store와 스레드의 vector_store를 모두 쿼리합니다.  

여기서 사용자는 Apple의 최신 10-K 파일을 추가로 첨부합니다. 

In [6]:
# OpenAI에 사용자 제공 파일 업로드
message_file = client.files.create(
  file=open("edgar/aapl-10k.pdf", "rb"), purpose="assistants"
)

# 스레드 생성 및 파일을 메시지에 첨부
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "2023년 10월 말에 AAPL의 발행 주식 수는 얼마였나요?",
      # 새 파일을 메시지에 첨부합니다.
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)

# 이제 스레드에는 파일이 있는 벡터 저장소가 툴 리소스로 있습니다.
print(thread.tool_resources.file_search)

ToolResourcesFileSearch(vector_store_ids=['vs_67a59fcdb7648191a4393900cf754c51'])


### 5단계: Run 만들기 및 출력 확인
이제Run행을 생성하고 모델이 파일 검색 도구를 사용하여 사용자의 질문에 대한 응답을 제공하는 것을 관찰합니다.

#### streaming 사용
OpenAI의 AssistantEventHandler를 사용하여 Run을 만들고 응답을 스트리밍할 수 있습니다.

In [7]:
from typing_extensions import override
from openai import AssistantEventHandler
 
# 먼저, 이벤트 핸들러 클래스(EventHandler)를 생성하여
# 응답 스트림에서 이벤트를 어떻게 처리할지 정의합니다.
class EventHandler(AssistantEventHandler):    
  # 텍스트 생성이 완료되었을 때 호출되는 메서드
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant-1 > ", end="", flush=True)
      
  # 텍스트 생성 중간에 호출되는 메서드
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  # 도구 호출이 생성되었을 때 호출되는 메서드
  @override
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant-2 > {tool_call.type}\n", flush=True)
  
  # 도구 호출 중간에 호출되는 메서드
  @override
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 
# 그런 다음, `stream` SDK 도우미와 `EventHandler` 클래스를 사용하여
# Run을 생성하고 응답을 스트리밍합니다.
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="사용자를 고객님이라고 부르세요. 사용자에게 프리미엄 계정이 있습니다.",  # 사용자를 고객님이라고 부르고, 프리미엄 계정을 가지고 있다고 지시
  event_handler=EventHandler(),  # EventHandler 클래스 인스턴스를 이벤트 핸들러로 사용
) as stream:
  stream.until_done()  # 스트림이 완료될 때까지 대기


assistant-2 > file_search


assistant-1 > 2023년 10월 말에 AAPL의 발행 주식 수는 15,943,425 주였습니다. 이는 2022년 9월 24일 기준으로 기록된 수치입니다. 추가적으로, 2022년에는 569백만 주가 재구매되었고, 그 결과로 발행 주식 수가 감소하였습니다【4:12†source】.

### Thread에 새로운 message 추가 및 Run 생성

In [8]:
# thread 에 사용자 message 추가
message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role="user",
      content="2023년 Google의 당기 순이익은 얼마였나요? BERKSHIRE HATHAWAY INC. 와 어느 쪽의 당기 순이익이 더 많았나요?"
    )

# OpenAI Assistant 실행 (Function Calling 및 대화 흐름 처리)
with client.beta.threads.runs.stream(
      thread_id=thread.id,            # 실행할 Thread(대화 세션) ID
      assistant_id=assistant.id,      # 실행할 Assistant ID
      instructions="사용자를 고객님이라고 부르세요. 사용자에게 프리미엄 계정이 있습니다.",  # Assistant의 추가 지시사항
      event_handler=EventHandler(),  # Function Calling 및 이벤트 처리를 담당할 핸들러
    ) as stream:
      stream.until_done()           # 모든 처리가 완료될 때까지 스트리밍 대기


assistant-2 > file_search


assistant-1 > 2023년 Google (알파벳)의 당기 순이익은 18,432백만 달러였습니다【10:2†source】. 반면, Berkshire Hathaway의 당기 순이익은 42,521백만 달러였습니다【10:14†source】.

따라서, Berkshire Hathaway의 당기 순이익이 Google보다 더 많습니다.